In [5]:
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2000", "endyear":"2023"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()
    

In [20]:
from collections import namedtuple
BLSLaborForce = namedtuple('BLSLaborForce', ['name', 'seasonal_adjustment', 'sample_frequency', 'unit', 'age_group', 'sex_group'])
bls_map = {
    'LNS14000000': ['unemployment_rate', 's.a.', 'monthly', 'percent', 'geq 16', 'all gender'],
    'LNS14000001': ['unemployment_rate', 's.a.', 'monthly', 'percent', 'geq 16', 'men'],
    'LNS14000002': ['unemployment_rate', 's.a.', 'monthly', 'percent', 'geq 16', 'women'],
}
for k, v in bls_map.items():
    bls_map[k] = BLSLaborForce(*v)

In [21]:
bls_map

{'LNS14000000': BLSLaborForce(name='unemployment_rate', seasonal_adjustment='s.a.', sample_frequency='monthly', unit='percent', age_group='geq 16', sex_group='all gender'),
 'LNS14000001': BLSLaborForce(name='unemployment_rate', seasonal_adjustment='s.a.', sample_frequency='monthly', unit='percent', age_group='geq 16', sex_group='men'),
 'LNS14000002': BLSLaborForce(name='unemployment_rate', seasonal_adjustment='s.a.', sample_frequency='monthly', unit='percent', age_group='geq 16', sex_group='women')}

In [18]:
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"1960", "endyear":"2023"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
output = dict()
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    output[seriesId] = []
    for item in series['data']:
        year = item['year']
        period = item['period']
        periodName = item['periodName']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            output[seriesId].append({'seriesId': seriesId, 'year': year, 'period': period, 
                                     'period_label': periodName, 'value': value, 'footnotes': footnotes[0:-1]})

In [19]:
import pandas as pd
pd.DataFrame(output['CUUR0000SA0'])

,seriesId,year,period,period_label,value,footnotes
0,CUUR0000SA0,1969,M12,December,37.7,
1,CUUR0000SA0,1969,M11,November,37.5,
2,CUUR0000SA0,1969,M10,October,37.3,
3,CUUR0000SA0,1969,M09,September,37.1,
4,CUUR0000SA0,1969,M08,August,37.0,
...,...,...,...,...,...,...
115,CUUR0000SA0,1960,M05,May,29.5,
116,CUUR0000SA0,1960,M04,April,29.5,
117,CUUR0000SA0,1960,M03,March,29.4,
118,CUUR0000SA0,1960,M02,February,29.4,
